# 提示词里模板中的模板

## MessagePromptTemplate

LangChain提供了不同类型的 MessagePromptTemplate 。最常用的是 AIMessagePromptTemplate 、 SystemMessagePromptTemplate 和 HumanMessagePromptTemplate ，它们分别创建 AI 消息、系统消息和人工消息。

但是，如果聊天模型支持使用任意角色获取聊天消息，则可以使用 ChatMessagePromptTemplate ，它允许用户指定角色名称。

In [1]:
from langchain_core.prompts import ChatMessagePromptTemplate

template = "愿{subject}与你同在！"

prompt = ChatMessagePromptTemplate.from_template(
    role="kkutys", template=template
)

print(prompt.format(subject="上帝"))

content='愿上帝与你同在！' additional_kwargs={} response_metadata={} role='kkutys'


LangChain 还提供了 MessagesPlaceholder ，它使您可以完全控制格式化期间要呈现的消息。当您不确定消息提示模板应使用什么角色或希望在格式化期间插入消息列表时，这会很有用。

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

human_prompt = "用{word_count}个字总结一下迄今为止我们之间的对话。"

human_message_prompt = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="conversation"),
    human_message_prompt
])

human_message = HumanMessage(content="学习炒股最好的方法是什么？")
ai_message = AIMessage(
    content="""\
        1. 首先，学习各类金融知识，了解各类财务指标的含义。
        2. 其次，学习各类财报的阅读和分析，找到适合自己的一种的财报解读方法论。
        3. 最后，学习1-2两种技术指标，可以参考一些经典技术书籍学习，找到适合自己的一种组合指标分析方法。
    """
)

print(chat_prompt.format_messages(
    conversation=[human_message, ai_message], 
    word_count=10
))


[HumanMessage(content='学习炒股最好的方法是什么？', additional_kwargs={}, response_metadata={}), AIMessage(content='        1. 首先，学习各类金融知识，了解各类财务指标的含义。\n        2. 其次，学习各类财报的阅读和分析，找到适合自己的一种的财报解读方法论。\n        3. 最后，学习1-2两种技术指标，可以参考一些经典技术书籍学习，找到适合自己的一种组合指标分析方法。\n    ', additional_kwargs={}, response_metadata={}), HumanMessage(content='用10个字总结一下迄今为止我们之间的对话。', additional_kwargs={}, response_metadata={})]


In [3]:
# 结合大模型分析
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("ZHIPUAI_API_KEY")
base_url = os.getenv("ZHIPUAI_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.9, max_tokens=8192, model="glm-4-plus")

output_parser = StrOutputParser()

chain = chat_prompt | llm | output_parser

print(chain.invoke({"conversation": [human_message, ai_message], "word_count": 50}))

探讨学习炒股方法，包括金融知识、财报分析和技术指标，强调系统学习和实践结合。


## 部分提示模板

与其他方法一样，“部分”提示模板是有意义的 - 例如传入所需值的子集，以创建一个仅需要剩余值子集的新提示模板。

LangChain 通过两种方式支持这一点： 
1. 使用字符串值进行部分格式化。 
2. 使用返回字符串值的函数进行部分格式化。

这两种不同的方式支持不同的用例。在下面的示例中，我们将回顾这两个用例的动机以及如何在 LangChain 中实现这一点。


### 部分带字符串

想要部分提示模板的一个常见用例是，如果您先于其他变量获取某些变量。例如，假设您有一个提示模板需要两个变量 foo 和 baz 。如果您在链的早期获得 foo 值，但稍后获得 baz 值，那么等到两个变量都位于同一位置才能将它们传递给提示模板。相反，您可以使用 foo 值部分化提示模板，然后传递部分化的提示模板并直接使用它。下面是执行此操作的示例：

In [4]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("foo: {foo} baz: {baz}")

partial_prompt = prompt.partial(foo="bar")

print(partial_prompt.format(baz="qux"))

foo: bar baz: qux


In [5]:
prompt = PromptTemplate(
    template="foo: {foo} baz: {baz}",
    partial_variables={"foo": "bar"},
    input_variables=["baz"]
)

print(prompt.format(baz="qux"))

foo: bar baz: qux


## 部分函数

另一个常见用途是对函数进行部分处理。这样做的用例是当您有一个变量时，您知道您总是希望以通用方式获取该变量。一个典型的例子是日期或时间。想象一下，您有一个提示，您总是希望获得当前日期。您无法在提示中对其进行硬编码，并且将其与其他输入变量一起传递有点烦人。在这种情况下，能够使用始终返回当前日期的函数来部分提示是非常方便的。

In [6]:
from datetime import datetime


def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")

prompt = PromptTemplate.from_template("今天是{date}, 请给我讲一个关于{topic}的经典论述.")

partial_prompt = prompt.partial(date=get_current_date)

print(partial_prompt.format(topic="爱情"))

今天是2024-12-22, 请给我讲一个关于爱情的经典论述.


In [7]:
# 结合大模型分析
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os


prompt = PromptTemplate(
    template="今天是{date}, 请给我讲一个关于{topic}的经典论述.",
    input_variables=["topic"],
    partial_variables={"date": get_current_date}
)

load_dotenv()

api_key = os.getenv("ZHIPUAI_API_KEY")
base_url = os.getenv("ZHIPUAI_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.9, max_tokens=8192, model="glm-4-plus")

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

for s in chain.stream({"topic": "量子力学"}):
    print(s, end="", flush=True)

当然可以。今天我将为你讲述一个关于量子力学的经典论述，即海森堡不确定性原理（Heisenberg Uncertainty Principle）。

### 海森堡不确定性原理

海森堡不确定性原理是量子力学中的一个核心概念，由德国物理学家维尔纳·海森堡于1927年提出。这一原理揭示了在量子尺度上，某些物理量（如位置和动量）无法同时被精确测量。

#### 基本内容

不确定性原理可以用以下数学表达式来描述：

\[ \Delta x \Delta p \geq \frac{\hbar}{2} \]

其中：
- \(\Delta x\) 表示位置 \(x\) 的不确定性。
- \(\Delta p\) 表示动量 \(p\) 的不确定性。
- \(\hbar\) 是约化普朗克常数，约为 \(1.054 \times 10^{-34} \, \text{J} \cdot \text{s}\)。

这个不等式表明，位置和动量的不确定性乘积的最小值是 \(\hbar/2\)，也就是说，我们不能同时精确地知道一个粒子的位置和动量。

#### 物理意义

1. **测量干扰**：在量子尺度上，测量过程本身会对被测量的系统产生影响。例如，为了测量一个电子的位置，我们需要用光子去撞击它，但这一过程会改变电子的动量。

2. **波粒二象性**：量子粒子既表现出波动性，也表现出粒子性。不确定性原理反映了这种波粒二象性的本质。波函数的 Fourier 变换关系也支持这一观点。

3. **非经典概率**：在量子力学中，粒子的状态由波函数描述，波函数的平方给出了粒子出现在某位置的概率。不确定性原理表明，这种概率性是内在的，而不是由于测量技术不足。

#### 经典例子

假设我们试图测量一个电子的位置和动量。如果我们使用高精度的仪器来测量电子的位置，那么电子的动量就会变得非常不确定；反之，如果我们精确测量电子的动量，那么其位置就会变得非常不确定。

### 影响和应用

海森堡不确定性原理不仅在理论物理学中具有重要意义，还在多个领域产生了深远影响：

1. **量子计算**：不确定性原理是量子比特（qubit）的基础，量子比特可以同时处于0和1的叠加态，这为量子计算提供了强大的并行处理能力。

2. **量子加密**：基于不确定性原理的量子密钥分发（QKD）技术，可以实现无法被破解的加